# Import

In [1]:
import csv
import os
import pickle
import sys

import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from tqdm import tqdm

<jemalloc>: MADV_DONTNEED does not work (memset will be used instead)
<jemalloc>: (This is the expected behaviour if you are running under QEMU)


## Add configuration file

In [2]:
sys.path.append("/home/jovyan/core/config/")

In [3]:
from ALL import config 

## Set condition

In [4]:
tqdm.pandas()
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 50)

# Read data

In [5]:
ag_news_df = pd.read_csv("../../../Preprocessing/data/AgNews/master.csv", index_col=0)

In [6]:
with open("../../../Preprocessing/data/AgNews/class.csv", mode="r") as f:
    reader = csv.reader(f)
    class_labels = [label for label in reader]

In [9]:
vector_dims = config["vectorize"]["doc2vec"]["dims"]
model_nums = config["clustering"]["kmeans"]["max_model_num"]
vector_model_num = config["vectorize"]["doc2vec"]["max_model_num"]

In [10]:
n_components = 4

In [11]:
def getKMeans(vectors, n_components, seed, path):
    model = KMeans(n_clusters=n_components, random_state=seed)
    model.fit(vectors)
    # save model
    os.makedirs(os.path.dirname(path), exist_ok=True)
    pickle.dump(model, open(path, "wb"))
    pred = model.predict(vectors)
    return pred

In [ ]:
vectors_path = "../../data/AgNews/doc2vec/vector/"
models_path = "../../data/AgNews/doc2vec/KMeans/model/"
pred_path = "../../data/AgNews/doc2vec/KMeans/pred/"
for vector_dim in tqdm(vector_dims):
    for model_num in range(model_nums):
        vector_path = f"{vectors_path}{vector_dims}/normalized/{vector_model_num}.csv"
        vectors = np.loadtxt(vector_path, delimiter=",")

        os.makedirs(f"{models_path}{vector_dims}", exist_ok=True)
        pred = getKMeans(
            vectors,
            seed=model_num,
            n_components=n_components,
            path=f"{models_path}{vector_dims}/{model_num}.sav",
        )

        # save prediction
        os.makedirs(f"{pred_path}{vector_dims}", exist_ok=True)
        with open(f"{pred_path}{vector_dims}/{model_num}.csv", "w") as f:
            writer = csv.writer(f)
            writer.writerow(pred)

 43%|████▎     | 6/14 [02:45<03:43, 27.96s/it]